9,16,25,29,41,74,96,91

In [14]:
import os
import numpy as np
import pandas as pd
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
import torch
import lpips
from clip import clip
import matplotlib.pyplot as plt
from torchvision import transforms
from PIL import Image

def get_optimal_win_size(max_size, min_dim):
    """
    Returns the largest odd win_size <= min(max_size, min_dim).
    """
    for ws in range(min(max_size, min_dim), 2, -1):
        if ws % 2 == 1:
            return ws
    return None

def calculate_metrics(hr_dir, models_dirs):
    results = []

    # LPIPS setup
    lpips_fn = lpips.LPIPS(net='alex')

    # CLIPIQA setup
    clip_model, preprocess = clip.load("ViT-B/32", device="cuda")

    for model_name, model_dir in models_dirs.items():
        model_psnr = []
        model_ssim = []
        model_lpips = []
        model_clip = []

        for img_name in os.listdir(model_dir):
            hr_img_path = os.path.join(hr_dir, img_name)
            gen_img_path = os.path.join(model_dir, img_name)

            if not os.path.exists(hr_img_path):
                continue

            # Load images
            hr_img = np.array(Image.open(hr_img_path).convert('RGB'))
            gen_img = np.array(Image.open(gen_img_path).convert('RGB'))
            print(f"HR Image Dimensions: {hr_img.shape}, Generated Image Dimensions: {gen_img.shape}")

            if hr_img.shape != gen_img.shape:
                print(f"Dimension mismatch: HR: {hr_img.shape}, Generated: {gen_img.shape}")
                common_height = min(hr_img.shape[0], gen_img.shape[0])
                common_width = min(hr_img.shape[1], gen_img.shape[1])

                # Crop both images
                hr_img = hr_img[:common_height, :common_width, :]
                gen_img = gen_img[:common_height, :common_width, :]

            # Calculate PSNR
            model_psnr.append(psnr(hr_img, gen_img))

            # Calculate SSIM
            min_dim = min(hr_img.shape[0], hr_img.shape[1])
            win_size = get_optimal_win_size(7, min_dim)
            if win_size is None:
                print(f"Skipping SSIM for {img_name}: Images too small for SSIM calculation.")
                continue  # Skip SSIM calculation for this image pair

            # Calculate SSIM with the determined win_size
            try:
                ssim_value = ssim(hr_img, gen_img, channel_axis=-1, win_size=win_size)
                model_ssim.append(ssim_value)
            except ValueError as ve:
                print(f"SSIM calculation failed for {img_name}: {ve}")
                continue

            print("SSIM Value:", ssim_value)

            # Calculate LPIPS
            hr_tensor = transforms.ToTensor()(Image.open(hr_img_path).convert('RGB')).unsqueeze(0).to('cuda')
            gen_tensor = transforms.ToTensor()(Image.open(gen_img_path).convert('RGB')).unsqueeze(0).to('cuda')
            model_lpips.append(lpips_fn(hr_tensor, gen_tensor).item())

            # Calculate CLIPIQA
            hr_clip_input = preprocess(Image.open(hr_img_path).convert('RGB')).unsqueeze(0).to('cuda')
            gen_clip_input = preprocess(Image.open(gen_img_path).convert('RGB')).unsqueeze(0).to('cuda')
            hr_features = clip_model.encode_image(hr_clip_input)
            gen_features = clip_model.encode_image(gen_clip_input)
            similarity = torch.cosine_similarity(hr_features, gen_features)
            model_clip.append(similarity.item())

        results.append({
            "Model": model_name,
            "PSNR": np.mean(model_psnr),
            "SSIM": np.mean(model_ssim),
            "LPIPS": np.mean(model_lpips),
            "CLIPIQA": np.mean(model_clip),
        })

    return results

# Directories
hr_dir = "dataset/DIV/selected/high"
models_dirs = {
    "BSRGAN": "flask/static/images/selected_256_BSRGAN",
    "RealESRGAN": "flask/static/images/selected_256_RealESRGAN",
    "ResShift": "flask/static/images/selected_256_ResShift",
    "SwinIR": "flask/static/images/selected_256_SwinIR",
}

# Calculate metrics
metrics = calculate_metrics(hr_dir, models_dirs)

# Convert to DataFrame
df = pd.DataFrame(metrics)

# Add summary row
summary = {
    "Model": "Summary",
    "PSNR": df['PSNR'].mean(),
    "SSIM": df['SSIM'].mean(),
    "LPIPS": df['LPIPS'].mean(),
    "CLIPIQA": df['CLIPIQA'].mean(),
}
df = df.append(summary, ignore_index=True)

# Save and display results
output_path = "metrics_comparison.csv"
df.to_csv(output_path, index=False)
print(df)


Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/home/hamzaz/miniconda3/envs/apw/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hamzaz/miniconda3/envs/apw/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/hamzaz/miniconda3/envs/apw/lib/python3.11/site-packages/lpips/weights/v0.1/alex.pth


/home/hamzaz/miniconda3/envs/apw/lib/python3.11/site-packages/lpips/lpips.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(model_path, m

HR Image Dimensions: (678, 1020, 3), Generated Image Dimensions: (676, 1020, 3)
Dimension mismatch: HR: (678, 1020, 3), Generated: (676, 1020, 3)
SSIM Value: 0.651444762639113


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!